使用 virtual environment 因ckip-transformers所需

In [1]:
import math
import os
import random
import re
import time
from collections import Counter
from contextlib import closing
from string import punctuation as enPunc

import gensim
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
# from ckiptagger import WS, POS, NER
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
from gensim.models.word2vec import Word2Vec
# from transformers import (BertTokenizerFast, AutoModel)
from opencc import OpenCC
from zhon.hanzi import punctuation as zhPunc

/opt/anaconda3/envs/tf15/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Target Page : V 2.0

- input: 某taskId包含之網址
- output: 網頁重新排序後之列表
---------------------
0. (載入)預訓練詞向量
1. 爬蟲&資料預處理：爬網頁文章內容、分詞並計算tfidf
2. 排序模組：匯入詞向量並進行增量訓練，取得搜尋詞詞向量，取得文件向量並與搜尋詞比較相似性

In [2]:
user_name = 'csti-user'
file_dir = 'OSINT'

### 爬蟲&資料預處理

#### 爬蟲取得網頁內容文章

In [3]:
def __get_content_texts(google_search_rs):
    """爬蟲取得網頁內容文章
    
    :param google_search_rs: 特定taskID中所有url
    :return context: 網頁文章，以句子分開
    """
    articleLength, context = [], []
    for idx, url in enumerate(google_search_rs):
#         print(idx +1, url)
        try:
            with closing(requests.get(url, headers={
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"
            }, timeout=30, verify=False)) as resp:
                if resp.ok and resp.headers.get("Content-Type", "").startswith("text/html"):
                    soup = BeautifulSoup(resp.content.decode("utf-8", "ignore"), "lxml")
                    if soup.body:
                        for tree in soup.find_all([
                            "a", "audio", "button", "head", "iframe", "img",
                            "input", "link", "meta", "noscript", "object", "script",
                            "select", "source", "style", "textarea", "title", "video"
                        ]):
                            tree.extract()
                        if soup.body.find("main"):
                            texts = [
                                re.sub(r"[\xa0\r\n\t\f\v ]+", " ", text.strip())
                                for text in soup.body.find("main").get_text().splitlines()
                                if re.search(r"\w", text)
                            ]
                        else:
                            texts = [
                                re.sub(r"[\xa0\r\n\t\f\v ]+", " ", text.strip())
                                for text in soup.body.get_text().splitlines()
                                if re.search(r"\w", text)
                            ]
            context.append(texts)
        except Exception as e:
            context.append([""])
            print(e)
        
        length = sum(len(s) for s in texts)          
        articleLength.append(length if length != 0 else 0)
    
    return context, articleLength

#### 分詞&計算詞頻

In [8]:
STOPWORD_PATH = f'/Users/{user_name}/Downloads/{file_dir}/stopwords-master/baidu_stopwords.txt'
__stopwords = list("\xa0\r\n\t\f\v ") + list(enPunc) + list(zhPunc)
__converter = OpenCC("s2twp.json")

with open(STOPWORD_PATH, "r") as f:
    lines = f.read().splitlines()
    __stopwords.extend(lines)
    for line in lines:
        __stopwords.append(__converter.convert(line))

In [10]:
__valid_tags = ["EVENT", "FAC", "GPE", "LANGUAGE", "LAW", "LOC", "NORP", "ORG", "PERSON", "PRODUCT", "WORK_OF_ART"]

### ckip-transformers ###
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
# model = AutoModel.from_pretrained('ckiplab/bert-base-chinese-ner')

### ckip-tagger's nlp tool ###
# Initialize drivers
# ws_driver = CkipWordSegmenter(level=3)
# pos_driver = CkipPosTagger(level=3)
ner_driver = CkipNerChunker(level=3) #實體辨識

# Initialize drivers with custom checkpoints
# ws = WS(f'/Users/{user_name}/Downloads/{file_dir}/ckiptagger_data')
# pos = POS(f'/Users/{user_name}/Downloads/{file_dir}/ckiptagger_data')
# ner = NER(f'/Users/{user_name}/Downloads/{file_dir}/ckiptagger_data')   # word_n = ner(word_s, word_p)


In [15]:
# google_search_rs = pd.read_excel(f'/Users/{user_name}/Downloads/{file_dir}/data/try_1112.xlsx').url.to_list()
# contexts, articleLength = __get_content_texts(google_search_rs)
# counters = __get_counters(contexts)
tfidfs = __get_tfidfs(counters,15)
tfidfs

,idx,word,score
0,1,伊萬卡,0.44266
1,1,資訊科技有限公司,0.22133
2,1,中共,0.20763
3,1,Twitter,0.15917
4,1,川普,0.14290
...,...,...,...
543,48,振華公司,0.05032
544,48,振華,0.04844
545,48,唐家婕,0.04190
546,48,沈伯洋,0.03886


In [6]:
def __get_counters(context):
    """取得各篇文章詞頻
    
    :param context: 所有網頁文章(list)
    :return counters: 各篇文章詞頻(list)
    """
    counters = []
    for idx, doc in enumerate(context):
#     print(idx)
        df = pd.DataFrame(columns=["word", "ner", "idx"])
        if len(doc) > 1:
            text = [__converter.convert(i) for i in doc]
            text_ner = ner_driver(text, use_delim=False)
            lst = [text_ner[k] for k in range(len(text_ner))]
            for k in range(len(lst)):
                df = df.append(lst[k], ignore_index=True)
            for eachrow in range(len(df)):
                if df.ner[eachrow] not in __valid_tags or df.word[eachrow] in __stopwords:
                    df = df.drop([eachrow])
            counter = Counter(df.word)
        else:
            counter = Counter("")
        counters.append(counter)
    return counters

#### 計算tfidf

In [13]:
def __get_tfidfs(counters, top):
    """取得各篇文章的關鍵字及其透過TF-IDF加權分數的前20名
    
    :param counters: 各篇文章詞頻
    :param top: 前?名
    :return 各篇文章的前?名關鍵詞(dataframe)
    """
    def tf(word, counter):
        return counter[word] / sum(counter.values())

    def idf(word, counters):
        return math.log2(len(counters) / sum(1 for counter in counters if word in counter))

    data = []
    for idx, counter in enumerate(counters):
        data.extend([idx, word, round(score, 5)] for word, score in sorted([
            (word, tf(word, counter) * idf(word, counters)) for word in counter
        ], key=lambda x: x[1], reverse=True)[:top])
    
    return pd.DataFrame(data, columns=["idx", "word", "score"])

### 排序模組

#### 匯入詞向量並進行增量訓練

In [9]:
MODEL_PATH = f'/Users/{user_name}/Downloads/{file_dir}/model/api_version/word2vec.model'
def __get_trained_model(query, corpus):
    """詞向量增量訓練

    :param query(str): 搜尋詞
    :param corpus: 語料庫，預期帶入文章分詞結果集合
    :return: 增量訓練後的模型
    """
    if os.path.isfile(MODEL_PATH): #該路徑是否為文件
        model = Word2Vec.load(MODEL_PATH)
    else:
        return None

    word_s = ws([query], sentence_segmentation=True, 
            segment_delimiter_set={'?', '？', '!', '！', '。', ',', '，', ';', ':', '、'})
    words = [word for word in word_s[0] if word != ' ']

    model.build_vocab(corpus, update=True, trim_rule=lambda word, count, min_count:
                      gensim.utils.RULE_KEEP if word in words else gensim.utils.RULE_DEFAULT)
    model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)
    model.save(MODEL_PATH)

    return model

#### 取得搜尋詞詞向量

In [10]:
def __get_vector(query, model):
    """取得搜尋詞及其向量

    若詞向量中有此搜尋詞，則取得該詞彙之向量，
    若詞向量中無此搜尋詞，則進行分詞，再尋找該詞彙之向量

    :param query(str): 搜尋詞
    :param model(Word2Vec): 詞向量的相關模型
    :return: vector
    """
    try:
        vector = model.wv.vectors[model.wv.key_to_index[query]]
    except:
        word_s = ws([query], sentence_segmentation=True, 
            segment_delimiter_set={'?', '？', '!', '！', '。', ',', '，', ';', ':', '、'})
        words = [word for word in word_s[0] if word != ' ']
        vectors = [
            model.wv.vectors[model.wv.key_to_index[word]]
            for word in words if word not in __stopwords
        ]
        vector = sum(vectors) / len(vectors)
    return vector

#### 取得文件向量並與搜尋詞比較相似性

In [11]:
# 因python 3.7 math 沒有dist 這個function
def get_dist(p, q):
    return math.sqrt(sum((px - qx) ** 2.0 for px, qx in zip(p, q)))

In [12]:
def __get_similarity(google_search_rs, top, query):
    contexts, articleLength = __get_content_texts(google_search_rs)
    counters = __get_counters(contexts)
    tfidfs = __get_tfidfs(counters,top) #所有網頁內容分詞的tfidfs，return dataframe
    model = __get_trained_model(query, contexts) #詞向量模型
    query_vector = __get_vector(query, model) #搜尋詞的詞向量
    
    vector = np.zeros(250)
    distance, words = [], []
    for idx in range(len(google_search_rs)):
        for row in tfidfs.loc[tfidfs["idx"] == idx].itertuples():
            try:
                vector += model.wv.vectors[model.wv.key_to_index[row.word]] * row.score
            except:
                continue
            else:
                words.append(row.word)
        else:
            total = len(words)
            distance.append(get_dist(vector if 0 == total else (vector / total), query_vector)) #各篇文章與搜尋詞的相似度

    for sort, idx in enumerate(
        sorted(range(len(google_search_rs)), key=lambda idx: distance[idx] 
               if 400 < articleLength[idx] else 100, reverse=False)):
        print("Article %d: %s"%(sort+1,google_search_rs[idx]))

## Info Extraction: V 2.0 (Api version)

In [11]:
def __get_keywords(counts, top):
    keyword_tfidfs = __get_tfidfs(counters,top)
    return keyword_tfidfs

## Target Page Demo

In [14]:
google_search_rs = pd.read_excel(f'/Users/{user_name}/Downloads/{file_dir}/data/try_1112.xlsx').url.to_list()
__get_similarity(google_search_rs, 15, "振華數據王雪峰情報")

1 https://www.bbc.com/zhongwen/trad/chinese-news-54160466


/opt/anaconda3/envs/tf15/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.bbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


2 https://www.epochtimes.com/gb/20/9/15/n12404579.htm


/opt/anaconda3/envs/tf15/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.epochtimes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


3 https://www.abc.net.au/chinese/2020-09-14/chinese-data-leak-linked-to-military-names-australians/12661676


...

49 https://www.taisounds.com/Global/Top-News/China/uid4739558758


/opt/anaconda3/envs/tf15/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.taisounds.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


50 http://ausnznet.com/news_sql/article_detail.asp?articleID=67613


Inference: 100%|██████████████████████████████████| 1/1 [00:07<00:00,  7.20s/it]


KeyError: '振華'

## Info Extraction Demo

In [ ]:
contexts, articleLength = __get_content_texts(google_search_rs)
counters = __get_counters(contexts)

In [ ]:
__get_keywords(counters, 15)[:20]